# __Fine-tuning a masked language model (PyTorch)__

## 0. Import, pip, ets.

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_path = "/content/drive/My Drive/Hugging_face/bert-finetuned-ner"

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForMaskedLM
from huggingface_hub import model_info
import torch
# from huggingface_hub import list_models

## 1. Загрузка модели и токенизатора

#### text

Для многих приложений NLP, включающих модели Transformer, годится предварительно обученная модель из Hugging Face.  
Однако иногда  нужно настроить языковые модели на ваших данных, прежде чем обучать конкретную для задачи голову. Например, если ваш набор данных содержит юридические контракты или научные статьи, ванильная модель Transformer, такая как BERT, обычно будет рассматривать доменно-специфические слова в вашем корпусе как редкие токены, и результирующая производительность может быть неудовлетворительной. Тонкая настройка языковой модели на внутридоменных данных позволяет повысить производительность!  
Этот процесс тонкой настройки предварительно обученной языковой модели на внутридоменных данных обычно называется адаптацией домена.

Хотя семейства моделей BERT и RoBERTa являются наиболее скачиваемыми, мы будем использовать модель под названием DistilBERT, которая может обучаться гораздо быстрее с небольшими потерями в производительности в дальнейшем. Эта модель была обучена с использованием специальной техники, называемой дистилляцией знаний, где большая «модель учителя», такая как BERT, используется для руководства обучением «модели ученика», которая имеет гораздо меньше параметров.  
Давайте продолжим и загрузим DistilBERT с помощью класса AutoModelForMaskedLM:

#### code

In [ ]:
# !git config --global user.email "you@example.com"
# !git config --global user.name "Your Name"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [ ]:
model_checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [ ]:
# Мы можем увидеть, сколько параметров имеет эта модель, вызвав метод num_parameters():
distilbert_num_parameters = model.num_parameters() / 1_000_000
print(f"'>>> DistilBERT number of parameters: {round(distilbert_num_parameters)}M'") # DistilBERT number of parameters: 67M (BERT  ~ 110M)

'>>> DistilBERT number of parameters: 67M'


In [ ]:
text = "This is a great [MASK]."

#### text

Как люди, мы можем представить себе множество возможностей для токена [MASK], например, «день», «езда» или «живопись». Для предварительно обученных моделей прогнозы зависят от корпуса, на котором была обучена модель, поскольку она учится улавливать статистические закономерности, присутствующие в данных. Как и BERT, DistilBERT был предварительно обучен на наборах данных английской Wikipedia и BookCorpus, поэтому мы ожидаем, что прогнозы для [MASK] будут отражать эти домены. Чтобы предсказать маску, нам нужен токенизатор DistilBERT для создания входных данных для модели, поэтому давайте загрузим его также из Hub:

#### code

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint) # превращает текст в тензор формата PyTorch (pt), который можно передавать в модель.

преобразовываем текст в тензор формата PyTorch (pt), подготовленный для передачи в модель.

In [ ]:
inputs = tokenizer(text, return_tensors="pt")
inputs

{'input_ids': tensor([[ 101, 2023, 2003, 1037, 2307,  103, 1012,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

Используя токенизатор и модель, передаем text в модель

In [ ]:
token_logits = model(**inputs).logits # Получение логитов (сырых вероятностей) токенов от модели:
# model(**inputs) — передает токенизированный текст в модель.
# .logits — извлекает логиты (до softmax), предсказанные моделью.

In [ ]:
token_logits.shape

torch.Size([1, 8, 30522])

#### text (__Формат torch.Size([batch_size, sequence_length, vocab_size])__

- `batch size` - (размер батча), означает, что в тензоре содержится один пример (один входной текст).  Если torch.Size([32, 8, 30522]) - 32 примера.  
— `sequence length` - (длина последовательности токенов), количество токенов в обработанном тексте. Например, "Hello, how are you?" получил 8 токенов.
- `vocab size` - размер словаря модели. Это количество возможных токенов в словаре модели. Например, для BERT-base стандартный размер словаря — 30522 токена.  
- Каждое значение в тензоре представляет логиты (сырые вероятности) для каждого токена из словаря.

__Как интерпретировать этот тензор?__
Этот тензор содержит логиты _для предсказания_ каждого токена в последовательности. _Для каждого из них есть вектор размером 30522_, где каждое значение соответствует вероятности (до softmax) выбора данного токена из словаря.

In [ ]:
token_logits[0][7]

tensor([-9.5213, -9.4632, -9.5022,  ..., -8.6561, -8.4908, -4.6903],
       grad_fn=<SelectBackward0>)

In [ ]:
token_logits[0][7].shape # Вектор логитов для 8-го токена в тексте

torch.Size([30522])

#### code

In [ ]:
# Find the location of [MASK] and extract its logits (text = "This is a great [MASK].")
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
print(mask_token_index)
print(tokenizer.mask_token_id)

tensor([5])
103


#### text (__Что делает код__ `mask_token_index = ...`  )

- `inputs["input_ids"]` — это числовые идентификаторы токенов входного текста.
```
{'input_ids': tensor([[ 101, 2023, 2003, 1037, 2307,  103, 1012,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}
```
- `tokenizer.mask_token_id` — ID маскированного токена ([MASK]). Когда в тексте присутствует [MASK], он заменяется специальным индексом, который можно получить через tokenizer.mask_token_id. В зависимости от используемого токенизатора (например, BERT, RoBERTa, DistilBERT) tokenizer.mask_token_id имеет фиксированное числовое значение. Например, для BERT - это 103, для DistilBERT (как у нас) - аналогично, 103. Когда этот тензор передается в модель: token_logits = model(**inputs).logits, модель предсказывает вероятности всех 30522 возможных токенов на месте 103 (то есть [MASK]). Дальше можно выбрать самые вероятные слова и подставить их вместо [MASK].  
- `torch.where(... == ...)` находит индекс (позицию) [MASK] в последовательности (то есть, у нас 8 токенов, на позициях 0, 1, 2, ..., и [MASK] у нас на позиции 5.

#### code

In [ ]:
mask_token_logits = token_logits[0, mask_token_index, :]
mask_token_logits.shape

torch.Size([1, 30522])

#### text (Что делает код: `mask_token_logits = ...` )


- `token_logits[0]` — логиты для первого (и единственного) элемента батча.  
- `mask_token_index` — индекс [MASK].  
- `:` — означает, что берем все предсказания для этого токена (логиты всех возможных слов).

#### code

In [ ]:
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist() # выбирает 5 наибольших логитов вдоль измерения 1 (по токенам).
# .indices[0].tolist() — берет их индексы и превращает в список.
print(top_5_tokens)
for token in top_5_tokens:
  print(tokenizer.decode([token]))
print("____________________")
for token in top_5_tokens:
  print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")
# tokenizer.decode([token]) - превращает токен обратно в слово.
# text.replace(tokenizer.mask_token, ...)` - заменяет [MASK] на предсказанное слово.

[3066, 3112, 6172, 2801, 8658]
deal
success
adventure
idea
feat
____________________
'>>> This is a great deal.'
'>>> This is a great success.'
'>>> This is a great adventure.'
'>>> This is a great idea.'
'>>> This is a great feat.'


Из результатов видно, что прогнозы модели относятся к повседневным терминам, что, возможно, неудивительно, учитывая основу английской Википедии. Давайте посмотрим, как мы можем изменить эту область на что-то более узкоспециализированное — высокополяризованные обзоры фильмов!

## 2. Адаптация (тонкая настройка) обученной на "обыденных" данных (Википедия) модели на выполнении специальной задачи

### Загрузка и просмотр данных

#### text

Будем использовать Large Movie Review Dataset (IMDb), который представляет собой корпус обзоров фильмов, часто используемый для сравнения моделей анализа настроений. Тонко настраивая DistilBERT на этом корпусе, мы ожидаем, что языковая модель адаптирует свой словарь из фактических данных Википедии, на которых она была предварительно обучена, к более субъективным элементам обзоров фильмов. Мы можем получить данные из Hugging Face Hub с помощью функции load_dataset() из 🤗 Наборы данных:

#### code

In [ ]:
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")
imdb_dataset # Видно, что обучающий и тестовый сплиты состоят из 25 000 отзывов, немаркированный сплит, неконтролируемый -  содержит 50 000 отзывов.

In [ ]:
# Рассмотрим несколько образцов из маркированных данных, создав случайную выборку и применив функции Dataset.shuffle() и Dataset.select()
sample = imdb_dataset["train"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"\n'>>> Review: {row['text']}'")
    print(f"'>>> Label: {row['label']}'")
    # видим, что 0 обозначает отрицательный отзыв, а 1 соответствует положительному.

In [ ]:
# Из немаркированных данных
sample = imdb_dataset["unsupervised"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"\n'>>> Review: {row['text']}'")
    print(f"'>>> Label: {row['label']}'")
    # видим, что у неразмеченных данных Label: -1' - что означает, что разметки нет.

### Предварительная обработка данных

#### text

Токенизируем наш корпус, но без установки параметра truncation=True в нашем токенизаторе.   
<small>(Как для авторегрессивного, так и для маскированного языкового моделирования общим этапом предварительной обработки является объединение всех примеров, а затем разделение всего корпуса _на фрагменты одинакового размера_, так как отдельные примеры могут быть усечены, если они слишком длинные, и это приведет к потере информации, которая может быть полезна для задачи языкового моделирования!)</small>  
Мы также захватим идентификаторы слов, если они доступны (что будет, если мы используем быстрый токенизатор, как описано в Главе 6), так как они нам понадобятся позже для маскировки целых слов. Мы обернем это в простую функцию, и удалим столбцы текста и меток, поскольку они нам больше не нужны:

<font color="yellow">Напомним, что у нас:
- model_checkpoint = "distilbert-base-uncased"
- model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
- tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)</font>


#### code

In [ ]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    # print(result["input_ids"])
    # print(len(result["input_ids"]))
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
        # print(result["word_ids"])
    return result

#### text (__result["word_ids"] = ...__)  

используется метод __word_ids()__, который доступен только для fast-токенизаторов (основанных на tokenizers из библиотеки Hugging Face).
Что делает эта строка?  
- `result["input_ids"]` — список токенов после токенизации.  
- `len(result["input_ids"])` — количество примеров в батче (сколько отдельных текстов было токенизировано).
- `result.word_ids(i)` -  возвращает список индексов исходных слов для каждого под-токена в i-той последовательности (i — номер примера в батче).
Если слово состоит из нескольких под-токенов, все эти под-токены будут привязаны к одному и тому же word_id.  
- `[result.word_ids(i) for i in range(len(result["input_ids"]))]` создается список, в котором для каждого примера в батче (i) вызывается result.word_ids(i).  
Итоговый список содержит списки word_ids для каждого текста.

Пример работы  
`text = ["I love NLP!", "Tokenization is fun."]`  
После токенизации (примерные input_ids):  
```
[
    [101, 1045, 2293, 17953, 999, 102],  # "I love NLP !"
    [101, 19204, 2003, 4569, 1012, 102]  # "Tokenization is fun ."
]  
```
[101] и [102] — специальные токены [CLS] и [SEP].  
"NLP" токенизируется в один токен 17953.  
"Tokenization" может разделиться на ["Token", "ization"].  
Результат word_ids():  
```
[
    [None, 0, 1, 2, 3, None],  # Для "I love NLP !"
    [None, 0, 1, 2, 3, None]   # Для "Tokenization is fun ."
]
```
None → специальные токены [CLS] и [SEP] (не относятся ни к одному слову).  
Остальные цифры (0, 1, 2, ...) → соответствуют индексам исходных слов:  
0 → "I"  
1 → "love"  
2 → "NLP"  
3 → "!"  
Аналогично для второго предложения.  
Зачем это нужно?  
word_ids позволяют отслеживать, к каким исходным словам относятся под-токены.
Это полезно для:  
Выравнивания аннотаций (например, при разметке Named Entity Recognition, NER).
Обратного преобразования токенов в слова.  
Анализа субтокенов (например, "Tokenization" разбивается на ["Token", "ization"], но относится к одному word_id).  
Вывод  
`result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]` добавляет в result список word_ids, который помогает отслеживать, какие под-токены относятся к каким исходным словам.  
Поскольку DistilBERT — это модель, подобная BERT, мы видим, что закодированные тексты состоят из `input_ids, attention_mask`, а также `word_ids`, которые мы добавили.  

#### code

Посмотрим, что делает ф-ция __tokenize_function__ (на примере 2-х примеров из датасета)

In [ ]:
# Use batched=True to activate fast multithreading!
tokenized_datasets_example = imdb_dataset["train"].select(range(1)).map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_datasets_example
# [[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, ...]]

Итак, добавим в result список word_ids, который помогает отслеживать, какие под-токены относятся к каким исходным словам.

In [ ]:
# Use batched=True to activate fast multithreading!
tokenized_datasets = imdb_dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_datasets

In [ ]:
tokenizer.model_max_length

512

#### text

Теперь, когда мы токенизировали наши обзоры фильмов, следующим шагом будет сгруппировать их все вместе и разбить результат на фрагменты. Это в конечном итоге будет определяться объемом памяти GPU, который у нас есть, и хорошей отправной точкой является максимальный размер контекста модели, который мы узнали выше (.model_max_length токенизатора из файла tokenizer_config.json, связанного с контрольной точкой. Видим, что размер контекста составляет 512 токенов, как и в BERT.

✏️ Некоторые модели Transformer, такие как BigBird и Longformer, имеют гораздо большую длину контекста, чем BERT и другие ранние модели Transformer.

Чтобы запустить наши эксперименты на графических процессорах, выберем что-то немного меньшее, что может поместиться в памяти:

#### code

In [ ]:
chunk_size = 128

Возьмем несколько отзывов из токенизированного обучающего набора и выведем количество токенов:

In [ ]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets["train"][45:50]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 174'
'>>> Review 1 length: 152'
'>>> Review 2 length: 149'
'>>> Review 3 length: 172'
'>>> Review 4 length: 163'


Затем мы можем объединить все эти примеры с помощью простого словарного понимания следующим образом:

In [30]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 810'


#### text (__как работает код k: sum(tokenized_samples[k], []) ...__)

Порядок выполнения кода
- `for k in tokenized_samples.keys()` - вытаскивает из каждого примера (их 5) — ключи ("input_ids", "attention_mask", "word_ids").
- `tokenized_samples[k]` — это список значений для каждого ключа k.
- `sum(tokenized_samples["input_ids"], [])` - объединяет их в один список,
- `{
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}` - формирует уже словарь, в котором ключи как раз этои "к", а значения - объединенные списки.  

Например:  
```
tokenized_samples = {
    "input_ids": [[1, 2, 3], [4, 5], [6, 7, 8]],
    "attention_mask": [[1, 1, 1], [1, 1], [1, 1, 1]]
}
```  
После выполнения:  
```
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
```  
получим:  
```
{
    "input_ids": [1, 2, 3, 4, 5, 6, 7, 8],
    "attention_mask": [1, 1, 1, 1, 1, 1, 1, 1]
}
```

#### code

Итак, общая длина проверена — разделим объединенные обзоры примера на фрагменты размера, заданного chunk_size. Результатом является словарь фрагментов для каждой функции:

In [ ]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

#### text

Как работает данный код  
1. Итерация по `concatenated_examples.items()`
Т.е., Берем каждую пару k, t для `input_ids`, `attention_mask` (`word_ids`), где k - ключ (это `input_ids`, `attention_mask` `word_ids`), t - список объединенных токенов.  
Для каждого t создаем список кусков.  
2. Используем range(0, total_length, chunk_size).  
3. Разрезаем t на куски по chunk_size.  
4. Формируем итоговый словарь chunks  
Записываем разрезанные последовательности по соответствующим ключам.

#### code

Последний фрагмент, как правило, меньше максимального размера фрагмента. Стратегии: отбросить, дополнить до chunk_size. Воспользуемся первым подходом. Обернем всю вышеприведенную логику в одну функцию и применим к токенизированным наборам данных:

In [32]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    # Создание столбца labels (копия столбца input_ids) связано с тем,
    # что в моделировании языка с масками цель состоит в том,
    # чтобы предсказать случайно замаскированные токены во входном пакете,
    # и, создавая столбец labels, мы предоставляем истину для обучения нашей языковой модели.
    return result

Теперь применим group_texts() к нашим токенизированным наборам данных с помощью функции Dataset.map():

<font color='yellow'>Важно! Мы смешиваем все примеры в кучу, поэтому их разделять теперь не получится (например, по приниципу положительный, нейтральный или отрицательный отзыв), но мы решаем задачу МАСКИРОВКИ!, в этом случае мы просто берем массив текстов.

In [ ]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Группировка и последующее разделение текстов на части дали гораздо больше примеров, чем наши исходные 25 000 для разделов обучения и тестирования. Это потому, что теперь у нас есть примеры, включающие смежные токены, которые охватывают несколько примеров из исходного корпуса. Можно увидеть это явно, посмотрев на специальные токены [SEP] и [CLS] в одном из фрагментов:

In [34]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

"as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years ago, this was considered pornographic. really, the sex and nudity scenes are few and far between, even then it ' s not shot like some cheaply made porno. while my countrymen mind find it shocking, in reality sex and nudity are a major staple in swedish cinema. even ingmar bergman,"

В этом примере вы можете увидеть два перекрывающихся обзора фильмов, один о фильме о старшей школе, а другой о бездомности. Давайте также посмотрим, как выглядят метки для моделирования замаскированного языка:

In [35]:
tokenizer.decode(lm_datasets["train"][1]["labels"])

"as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years ago, this was considered pornographic. really, the sex and nudity scenes are few and far between, even then it ' s not shot like some cheaply made porno. while my countrymen mind find it shocking, in reality sex and nudity are a major staple in swedish cinema. even ingmar bergman,"

Как и ожидалось от нашей функции group_texts() выше, это выглядит идентично декодированным input_ids — _но как тогда наша модель может чему-то научиться?_ Мы упускаем ключевой шаг: вставку токенов [MASK] в случайные позиции во входных данных! Давайте посмотрим, как мы можем сделать это на лету во время тонкой настройки с помощью специального сортировщика данных.

###Тонкая настройка DistilBERT с помощью API Trainer  

#### text

Тонкая настройка маскированной языковой модели почти идентична тонкой настройке модели классификации последовательностей. Единственное отличие состоит в том, что нам нужен специальный сортировщик данных, который может случайным образом маскировать некоторые токены в каждой партии текстов. К счастью, 🤗 Transformers поставляется с выделенным `DataCollatorForLanguageModeling` именно для этой задачи. Нам просто нужно передать ему токенизатор и аргумент mlm_probability, который указывает, какую долю токенов следует маскировать. Мы выберем 15%, что является суммой, используемой для BERT, и распространенным выбором в литературе:

#### code

<font color='yellow'>Важно!, мы имеем датасет с текстами оценок фильмов и мы хотим исходную предобученную модель, обученную на Википедии, использовать для прогнозов [MASK] (т.е. под специфическую задачу, НЕ выделение постов в определенные категории, а заполнять маску!), и при этом для заполнения маски под специфичную категорию постов, связанных с оценкой фильмов. Именно под эту задачу мы производим тонкую настройку исходной модели.</font>

In [41]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [42]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] i rented i am curious - [MASK] [MASK] my video store because of all the controversy that surrounded it when itᄏ first released in 1967. i also heard that at [MASK] it was seized by u. s. customs if it ever tried to enter this country complete therefore being [MASK] fan of films considered " controversial " i really had to see this for [MASK]. < [MASK] / > < br / > overseas plot is centered around a young swedish drama student named lena who wants to learn everything she [MASK] about [MASK]. in [MASK] she [MASK] [MASK] focus her attentions feedback making some sort [MASK] documentary inorganic what the average swede thought about [MASK] political issues such'

'>>> as the vietnam war and race [MASK] in the united states. [MASK] between asking politicians and ordinary [MASK]izens of stockholm about their opinions on politics, she has [MASK] with [MASK] drama teacher, classmates [MASK] and married men. < br / > [MASK] br / > what kills [MASK] about i [MASK] curious - yellow is

#### text

Отлично, сработало! Мы видим, что токен [MASK] был случайным образом вставлен в разные места нашего текста. Это будут токены, которые наша модель должна будет предсказать во время обучения — и прелесть сортировщика данных в том, что он будет рандомизировать вставку [MASK] с каждой партией!

✏️ Попробуйте! Запустите фрагмент кода выше несколько раз, чтобы увидеть, как случайная маскировка происходит прямо у вас на глазах! Также замените метод tokenizer.decode() на tokenizer.convert_ids_to_tokens(), чтобы увидеть, что иногда маскируется только один токен из заданного слова, а не другие.

Одним из побочных эффектов случайной маскировки является то, что наши метрики оценки не будут детерминированными при использовании Trainer, поскольку мы используем один и тот же сортировщик данных для обучающего и тестового наборов. Позже, когда мы рассмотрим тонкую настройку с помощью 🤗 Accelerate, мы увидим, как мы можем использовать гибкость пользовательского цикла оценки, чтобы заморозить случайность.

При обучении моделей для моделирования языка с масками можно использовать один из методов — маскировать целые слова вместе, а не только отдельные токены. Этот подход называется маскировкой целых слов. Если мы хотим использовать маскировку целых слов, нам нужно будет самостоятельно построить сортировщик данных. Коллектор данных — это просто функция, которая берет список образцов и преобразует их в пакет, так что давайте сделаем это сейчас! Мы будем использовать идентификаторы слов, вычисленные ранее, чтобы создать карту между индексами слов и соответствующими токенами, затем случайным образом решим, какие слова маскировать, и применим эту маску к входным данным. Обратите внимание, что все метки равны -100, за исключением тех, которые соответствуют словам маски.

#### code

In [43]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

Далее мы можем опробовать это на тех же образцах, что и раньше:

In [45]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")
    print(f"\n'>>> {tokenizer.convert_ids_to_tokens(chunk)}'")


'>>> [CLS] i rented i am curious - yellow from my video store because of all the controversy that surrounded [MASK] [MASK] it [MASK] first released in 1967. i also [MASK] [MASK] at first it was [MASK] [MASK] u [MASK] s. customs if [MASK] ever [MASK] to enter this country, [MASK] being [MASK] fan of films considered " controversial " i really had [MASK] see this for myself [MASK] < br / [MASK] < [MASK] / > the plot is centered around a young swedish drama student [MASK] [MASK] who wants [MASK] learn everything she can about life. [MASK] particular she wants to focus her attentions to making [MASK] sort of documentary [MASK] what the average swede [MASK] about certain political issues such'

'>>> ['[CLS]', 'i', 'rented', 'i', 'am', 'curious', '-', 'yellow', 'from', 'my', 'video', 'store', 'because', 'of', 'all', 'the', 'controversy', 'that', 'surrounded', '[MASK]', '[MASK]', 'it', '[MASK]', 'first', 'released', 'in', '1967', '.', 'i', 'also', '[MASK]', '[MASK]', 'at', 'first', 'it', 'wa

✏️ Попробуйте! Запустите фрагмент кода выше несколько раз, чтобы увидеть, как случайная маскировка происходит прямо у вас на глазах! Также замените метод tokenizer.decode() на tokenizer.convert_ids_to_tokens(), чтобы увидеть, что токены из заданного слова всегда маскируются вместе.

Теперь, когда у нас есть два сортировщика данных, остальные шаги тонкой настройки стандартны. Обучение может занять некоторое время в Google Colab, если вам не повезет получить мифический графический процессор P100 😭, поэтому сначала мы уменьшим размер обучающего набора до нескольких тысяч примеров. Не волнуйтесь, мы все равно получим довольно приличную языковую модель! Быстрый способ уменьшить выборку набора данных в 🤗 Datasets — это функция Dataset.train_test_split(), которую мы видели в Главе 5:

In [46]:
train_size = 10_000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1000
    })
})

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

Укажем аргументы для Trainer:

In [53]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    logging_steps=logging_steps,
    output_dir=f"{model_name}-finetuned-imdb",
    report_to="none"  # Отключает wandb
    # overwrite_output_dir=True,
    # push_to_hub=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Здесь мы подправили несколько параметров по умолчанию, включая logging_steps, чтобы гарантировать отслеживание потерь обучения с каждой эпохой. Мы также использовали fp16=True, чтобы включить обучение со смешанной точностью, что дает нам еще один прирост скорости. По умолчанию Trainer удалит все столбцы, которые не являются частью метода forward() модели.  
__Это означает, что если вы используете сортировщик с маскировкой целых слов, вам также нужно будет установить `remove_unused_columns=False`, чтобы гарантировать, что мы не потеряем столбец word_ids во время обучения.__

Обратите внимание, что вы можете указать имя репозитория, в который вы хотите отправить данные, с помощью аргумента hub_model_id (в частности, вам придется использовать этот аргумент для отправки в организацию). Например, когда мы отправили модель в организацию huggingface-course, мы добавили hub_model_id="huggingface-course/distilbert-finetuned-imdb" в TrainingArguments. По умолчанию используемый репозиторий будет находиться в вашем пространстве имен и назван в честь выходного каталога, который вы установили, поэтому в нашем случае это будет "lewtun/distilbert-finetuned-imdb".

Теперь у нас есть все ингредиенты для создания экземпляра Trainer. Здесь мы просто используем стандартный data_collator, но вы можете попробовать целословный маскирующий collator и сравнить результаты в качестве упражнения:

In [54]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    processing_class=tokenizer,
)

Теперь мы готовы запустить trainer.train(), но перед этим давайте кратко рассмотрим перплексию, которая является распространенной метрикой для оценки производительности языковых моделей.

В отличие от других задач, таких как классификация текста или ответы на вопросы, где нам дается помеченный корпус для обучения, при языковом моделировании у нас нет никаких явных меток. Так как же нам определить, что делает языковую модель хорошей? Как и в случае с функцией автозамены в вашем телефоне, хорошая языковая модель — это та, которая назначает высокие вероятности предложениям, которые грамматически правильны, и низкие вероятности бессмысленным предложениям. Чтобы дать вам лучшее представление о том, как это выглядит, вы можете найти целые наборы «неудач автозамены» в Интернете, где модель в телефоне человека выдала некоторые довольно забавные (и часто неуместные) завершения!

Предполагая, что наш тестовый набор состоит в основном из предложений, которые грамматически правильны, тогда один из способов измерить качество нашей языковой модели — это вычислить вероятности, которые она назначает следующему слову во всех предложениях тестового набора. Высокая вероятность указывает на то, что модель не «удивлена» и не «озадачена» невидимыми примерами, и предполагает, что она усвоила основные закономерности грамматики языка. Существуют различные математические определения озадаченности, но то, которое мы будем использовать, определяет ее как экспоненту потери кросс-энтропии. Таким образом, мы можем вычислить озадаченность нашей предварительно обученной модели, используя функцию Trainer.evaluate() для вычисления потери кросс-энтропии на тестовом наборе, а затем взяв экспоненту результата:

In [55]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 21.94


Более низкий показатель perplexity означает лучшую языковую модель, и мы можем видеть здесь, что наша начальная модель имеет довольно большое значение. Давайте посмотрим, сможем ли мы понизить его с помощью тонкой настройки! Для этого мы сначала запускаем цикл обучения:

In [56]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time
1,2.683800,2.509436,0.001700
2,2.587800,2.450192,0.001700
3,2.527900,2.481931,0.001700


TrainOutput(global_step=471, training_loss=2.599245999775621, metrics={'train_runtime': 153.9359, 'train_samples_per_second': 194.886, 'train_steps_per_second': 3.06, 'total_flos': 994208670720000.0, 'train_loss': 2.599245999775621, 'epoch': 3.0})

и затем вычислить результирующую сложность на тестовом наборе, как и раньше:

In [57]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 12.05


Отлично — это значительное снижение недоумения, которое говорит нам, что модель узнала что-то о области обзоров фильмов!

#### Сохранение модели

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [60]:
file_path = "/content/drive/My Drive/Hugging_face/distilbert-base-uncased_fine_tuning"

In [61]:
# trainer.push_to_hub()
output_dir = file_path  # Укажите путь для сохранения
trainer.save_model(output_dir)  # Сохранит модель и конфиг
tokenizer.save_pretrained(output_dir)  # Сохранит токенизатор

('/content/drive/My Drive/Hugging_face/distilbert-base-uncased_fine_tuning/tokenizer_config.json',
 '/content/drive/My Drive/Hugging_face/distilbert-base-uncased_fine_tuning/special_tokens_map.json',
 '/content/drive/My Drive/Hugging_face/distilbert-base-uncased_fine_tuning/vocab.txt',
 '/content/drive/My Drive/Hugging_face/distilbert-base-uncased_fine_tuning/added_tokens.json',
 '/content/drive/My Drive/Hugging_face/distilbert-base-uncased_fine_tuning/tokenizer.json')

Чтобы загрузить модель:  
```
from transformers import AutoModelForSequenceClassification
trainer.model = AutoModelForSequenceClassification.from_pretrained(file_path)
```


#### code

✏️ Ваша очередь! Запустите обучение выше, изменив сортировщик данных на сортировщик с маскировкой целых слов. Вы получаете лучшие результаты?

_____________________________
_____________________________

In [70]:
training_args_2 = TrainingArguments(
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    logging_steps=logging_steps,
    output_dir=f"{model_name}-finetuned-imdb",
    report_to="none",  # Отключает wandb
    remove_unused_columns=False, # Изменения
    # overwrite_output_dir=True,
    # push_to_hub=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [71]:
trainer_2 = Trainer(
    model=model,
    args=training_args_2,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=whole_word_masking_data_collator, # Изменения
    processing_class=tokenizer,
)

In [72]:
trainer_2.train()

Epoch,Training Loss,Validation Loss
1,3.398800,3.291717
2,3.325600,3.295487
3,3.290800,3.252086


TrainOutput(global_step=471, training_loss=3.3389588738702667, metrics={'train_runtime': 156.6676, 'train_samples_per_second': 191.488, 'train_steps_per_second': 3.006, 'total_flos': 994208670720000.0, 'train_loss': 3.3389588738702667, 'epoch': 3.0})

In [73]:
eval_results_2 = trainer_2.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 12.05


То есть, хотя мы и использовали "сортировщик" на целые слова, результат не улучшился. Попробуем увеличить обучающий набор.  

In [ ]:
train_size_2 = 20_000
test_size_2 = int(0.1 * train_size)

downsampled_dataset_2 = lm_datasets["train"].train_test_split(
    train_size=train_size_2, test_size=test_size_2, seed=42
)
downsampled_dataset_2

In [76]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset_2["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args_3 = TrainingArguments(
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    logging_steps=logging_steps,
    output_dir=f"{model_name}-finetuned-imdb",
    report_to="none",  # Отключает wandb
    remove_unused_columns=False, # Изменения
    # overwrite_output_dir=True,
    # push_to_hub=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [77]:
trainer_3 = Trainer(
    model=model,
    args=training_args_3,
    train_dataset=downsampled_dataset_2["train"],
    eval_dataset=downsampled_dataset_2["test"],
    data_collator=whole_word_masking_data_collator, # Изменения
    processing_class=tokenizer,
)

In [78]:
trainer_3.train()

Epoch,Training Loss,Validation Loss
1,3.312400,3.200478
2,3.250400,3.190410
3,3.222800,3.160626


TrainOutput(global_step=939, training_loss=3.261841346407597, metrics={'train_runtime': 326.5069, 'train_samples_per_second': 183.763, 'train_steps_per_second': 2.876, 'total_flos': 1988417341440000.0, 'train_loss': 3.261841346407597, 'epoch': 3.0})

In [79]:
eval_results_3 = trainer_3.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 12.05


Не улучшается ...........

В нашем случае нам не нужно было делать ничего особенного с циклом обучения, но в некоторых случаях вам может потребоваться реализовать некоторую пользовательскую логику. Для этих приложений вы можете использовать 🤗 Accelerate — давайте посмотрим!

### Тонкая настройка DistilBERT с помощью 🤗 Accelerate

Как мы видели с Trainer, тонкая настройка маскированной языковой модели очень похожа на пример классификации текста из главы 3. Фактически, единственная тонкость — это использование специального сортировщика данных, и мы уже рассмотрели это ранее в этом разделе!

Однако мы увидели, что DataCollatorForLanguageModeling также применяет случайную маскировку при каждой оценке, поэтому мы увидим некоторые колебания в наших оценках озадаченности при каждом запуске обучения. Один из способов устранить этот источник случайности — применить маскировку один раз ко всему тестовому набору, а затем использовать сортировщик данных по умолчанию в 🤗 Transformers для сбора пакетов во время оценки. Чтобы увидеть, как это работает, давайте реализуем простую функцию, которая применяет маскировку к пакету, аналогично нашей первой встрече с DataCollatorForLanguageModeling:

In [80]:
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = data_collator(features)
    # Create a new "masked" column for each column in the dataset
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}

Далее мы применим эту функцию к нашему тестовому набору и удалим немаскированные столбцы, чтобы заменить их замаскированными. Вы можете использовать маскировку целых слов, заменив data_collator выше на соответствующий, в этом случае вам следует удалить первую строку здесь:

In [81]:
downsampled_dataset = downsampled_dataset.remove_columns(["word_ids"])
eval_dataset = downsampled_dataset["test"].map(
    insert_random_mask,
    batched=True,
    remove_columns=downsampled_dataset["test"].column_names,
)
eval_dataset = eval_dataset.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_attention_mask": "attention_mask",
        "masked_labels": "labels",
    }
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Затем мы можем настроить загрузчики данных как обычно, но для оценочного набора мы будем использовать default_data_collator из 🤗 Transformers:

In [82]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

batch_size = 64
train_dataloader = DataLoader(
    downsampled_dataset["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)
eval_dataloader = DataLoader(
    eval_dataset, batch_size=batch_size, collate_fn=default_data_collator
)

Форма здесь, мы следуем стандартным шагам с 🤗 Accelerate. Первым делом нужно загрузить свежую версию предварительно обученной модели:

In [83]:
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Затем нам нужно указать оптимизатор, будем использовать стандартный AdamW:

In [84]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

С помощью этих объектов мы теперь можем подготовить все для обучения с помощью объекта Accelerator:

In [85]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

Теперь, когда наша модель, оптимизатор и загрузчики данных настроены, мы можем указать планировщик скорости обучения следующим образом:

In [86]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
# from huggingface_hub import get_full_repo_name

# model_name = "distilbert-base-uncased-finetuned-imdb-accelerate"
# repo_name = get_full_repo_name(model_name)
# repo_name

'lewtun/distilbert-base-uncased-finetuned-imdb-accelerate'

In [ ]:
# from huggingface_hub import Repository

# output_dir = model_name
# repo = Repository(output_dir, clone_from=repo_name)

Подготовим путь для записи обученной модели

In [88]:
file_path_2 = "/content/drive/My Drive/Hugging_face/distilbert-base-uncased_accelerate"

После этого останется только написать полный цикл обучения и оценки:

In [89]:
from tqdm.auto import tqdm
import torch
import math

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    print(f">>> Epoch {epoch}: Perplexity: {perplexity}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(file_path_2, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(file_path_2)

  0%|          | 0/471 [00:00<?, ?it/s]

>>> Epoch 0: Perplexity: 10.88425165507281
>>> Epoch 1: Perplexity: 10.670057089700505
>>> Epoch 2: Perplexity: 10.670057089700505


Здорово, нам удалось оценить сложность в каждой эпохе и гарантировать воспроизводимость множественных тренировочных запусков!

### Использование нашей тонко настроенной модели

Загрузим модель с помощью конвейера fill-mask:

In [90]:
from transformers import pipeline

mask_filler = pipeline("fill-mask", model=file_path_2)

Device set to use cuda:0


In [91]:
# text = "This is a great [MASK]."
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> this is a great film.
>>> this is a great movie.
>>> this is a great idea.
>>> this is a great one.
>>> this is a great story.


Our model has clearly adapted its weights to predict words that are more strongly associated with movies!